In [1]:
# Dependencies
import pandas as pd
#pd.options.mode.chained_assignment = None  # default='warn'

import json
import numpy as np

import re

from sqlalchemy import create_engine
import psycopg2

# pull in postgres password
import sys
sys.path.append("C:/Users/laura/bootcamp/")
from api_keys import db_password

# ZIP CODE DATA

In [2]:
# Store filepath in a variable
Zipcode = "Resources/Portland_Zip_and_Neighborhood.csv"

In [96]:
# Read our Data file with the pandas library
# Not every CSV requires an encoding, but be aware this can come up
zip_df = pd.read_csv(Zipcode, encoding="ISO-8859-1")
zip_df.head()

,Zip_Code,Neighborhood_Name
0,97212,Alameda
1,97217,Arbor Lodge
2,97222,Ardenwald
3,97230,Argay
4,97201,Arlington Heights


# AIRBNB DATA (no longer using but didn't want to delete code yet - Laura 4.7.22)

In [97]:
# Store filepath in a variable
ARBNB_old = "Resources/AB_US_2020.csv"

In [98]:
# Read our Data file with the pandas library
# Not every CSV requires an encoding, but be aware this can come up
ARBNB_df_old = pd.read_csv(ARBNB_old, encoding="ISO-8859-1")

C:\Users\laura\anaconda3\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [99]:
# Show just the header
#ARBNB_df_old.head()

In [100]:
column_list = ARBNB_df_old.columns.values.tolist()
#column_list

In [101]:
#ARBNB_df_old.describe()

In [102]:
# Keep the rows that are Portland
ARBNB_Portland_filter_old=ARBNB_df_old.loc[(ARBNB_df_old["city"]=="Portland")]
#ARBNB_Portland_filter_old.head()

In [103]:
#determine if there are blank data points that need to be dropped
#ARBNB_Portland_filter_old.isna().sum()

In [104]:
# drop columns that do not aid in analysis
ARBNB_Portland_old=ARBNB_Portland_filter_old.drop(columns=['ï»¿id','name','host_id','host_name',"neighbourhood_group","last_review"])
#ARBNB_Portland_old

In [105]:
#determine if there are blank data points that need to be dropped
#ARBNB_Portland_old.isna().sum()

In [106]:
#decided to drop nan values from reviews_per_month, could also look at dropping column
#ARBNB_Portland_dropna_old=ARBNB_Portland_old.dropna()

In [107]:
#ARBNB_Portland_dropna_old.isna().sum()

# AIRBNB DATA FROM AIRBNB SITE

In [178]:
# Store filepath in a variable
ARBNB = "Resources/Portland_airbnb_InsideAirbnb.csv"

In [179]:
#currently not using this function
def string_to_float(x):
    return float(x.strip('%'))/100

In [180]:
# Read our Data file with the pandas library
# Not every CSV requires an encoding, but be aware this can come up
ARBNB_df = pd.read_csv(ARBNB, encoding="ISO-8859-1", 
                       #converters={"host_response_rate" : string_to_float} - error won't let do on N/A
                      )

In [181]:
# Show just the header
ARBNB_df.head()

,id,listing_url,scrape_id,last_scraped,name,description,neighborhood_overview,picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,12899,https://www.airbnb.com/rooms/12899,20211220005848,2021-12-20,"Alberta Arts 2 bedroom suite, charming 1906 house","Please know that we are fully vaccinated, will...",We're within walking distance of a grocery and...,https://a0.muscache.com/pictures/225005/cf0083...,49682,https://www.airbnb.com/users/show/49682,Ali And David,2009-10-29,"Portland, Oregon, United States","We enjoy cooking, singing, hiking, bicycles, t...",within an hour,100%,100%,t,https://a0.muscache.com/im/users/49682/profile...,https://a0.muscache.com/im/users/49682/profile...,Concordia,1,1,"['email', 'phone', 'facebook', 'reviews', 'kba']",t,f,"Portland, Oregon, United States",Concordia,NaN,45.56488,-122.63418,Entire rental unit,Entire home/apt,3,NaN,1 bath,2.0,2.0,"[""Portable fans"", ""Security cameras on propert...",$89.00,3,730,3.0,3.0,1125.0,1125.0,3.0,1125.0,NaN,t,22,52,82,353,2021-12-20,591,40,1,2010-01-24,2021-11-30,4.93,4.94,4.98,4.99,4.99,4.93,4.94,14-218887-000-00-HO,t,1,1,0,0,4.08
1,37676,https://www.airbnb.com/rooms/37676,20211220005848,2021-12-20,Mt. Hood View in the Pearl District,"This 1,000 SF loft is located in the heart of ...",The Pearl district enjoys a walkability score ...,https://a0.muscache.com/pictures/212298/16fb6b...,162158,https://www.airbnb.com/users/show/162158,Paul,2010-07-09,"Portland, Oregon, United States",I look forward to helping people discover the ...,within a day,100%,55%,t,https://a0.muscache.com/im/users/162158/profil...,https://a0.muscache.com/im/users/162158/profil...,Pearl,2,2,"['email', 'phone', 'facebook', 'reviews', 'kba']",t,t,"Portland, Oregon, United States",Pearl,NaN,45.52564,-122.68273,Entire loft,Entire home/apt,3,NaN,1 bath,1.0,1.0,"[""Single level home"", ""Coffee maker"", ""Elevato...",$140.00,30,730,30.0,30.0,730.0,730.0,30.0,730.0,NaN,t,30,60,90,365,2021-12-20,127,4,1,2010-07-22,2021-12-01,4.88,4.86,4.86,4.83,4.77,4.94,4.66,NaN,f,1,1,0,0,0.91
2,41601,https://www.airbnb.com/rooms/41601,20211220005848,2021-12-20,Grandpa's Bunkhouse-Backyard Studio,Our quiet backyard cabin: Vaulted ceiling. Fre...,Ours is a neighborhood just off Noodle Flat in...,https://a0.muscache.com/pictures/294ac44c-168d...,179045,https://www.airbnb.com/users/show/179045,Jean,2010-07-24,"Portland, Oregon, United States",Jean is all about thrifty sustainable living a...,within an hour,100%,100%,t,https://a0.muscache.com/im/pictures/user/480c9...,https://a0.muscache.com/im/pictures/user/480c9...,Roseway,2,2,"['email', 'phone', 'reviews', 'kba']",t,t,"Portland, Oregon, United States",Roseway,NaN,45.54804,-122.58541,Entire residential home,Entire home/apt,2,NaN,1 bath,1.0,1.0,"[""Coffee maker"", ""Cooking basics"", ""Free stree...",$129.

In [182]:
ARBNB_df.dtypes

id                                                int64
listing_url                                      object
scrape_id                                         int64
last_scraped                                     object
name                                             object
                                                 ...   
calculated_host_listings_count                    int64
calculated_host_listings_count_entire_homes       int64
calculated_host_listings_count_private_rooms      int64
calculated_host_listings_count_shared_rooms       int64
reviews_per_month                               float64
Length: 74, dtype: object

In [183]:
column_list = ARBNB_df.columns.values.tolist()
column_list

['id',
 'listing_url',
 'scrape_id',
 'last_scraped',
 'name',
 'description',
 'neighborhood_overview',
 'picture_url',
 'host_id',
 'host_url',
 'host_name',
 'host_since',
 'host_location',
 'host_about',
 'host_response_time',
 'host_response_rate',
 'host_acceptance_rate',
 'host_is_superhost',
 'host_thumbnail_url',
 'host_picture_url',
 'host_neighbourhood',
 'host_listings_count',
 'host_total_listings_count',
 'host_verifications',
 'host_has_profile_pic',
 'host_identity_verified',
 'neighbourhood',
 'neighbourhood_cleansed',
 'neighbourhood_group_cleansed',
 'latitude',
 'longitude',
 'property_type',
 'room_type',
 'accommodates',
 'bathrooms',
 'bathrooms_text',
 'bedrooms',
 'beds',
 'amenities',
 'price',
 'minimum_nights',
 'maximum_nights',
 'minimum_minimum_nights',
 'maximum_minimum_nights',
 'minimum_maximum_nights',
 'maximum_maximum_nights',
 'minimum_nights_avg_ntm',
 'maximum_nights_avg_ntm',
 'calendar_updated',
 'has_availability',
 'availability_30',
 'availa

In [184]:
# keep columns to be used in analysis
ARBNB_df2=ARBNB_df[[
     #'id',
     'listing_url',
     #'scrape_id',
     #'last_scraped',
     #'name',
     #'description',
     #'neighborhood_overview',
     #'picture_url',
     #'host_id',
     #'host_url',
     #'host_name',
     #'host_since',
     #'host_location',
     #'host_about',
     'host_response_time',
     'host_response_rate',
     'host_acceptance_rate',
     'host_is_superhost',
     #'host_thumbnail_url',
     #'host_picture_url',
     #'host_neighbourhood',
     #'host_listings_count',
     #'host_total_listings_count',
     #'host_verifications',
     #'host_has_profile_pic',
     'host_identity_verified',
     #'neighbourhood',
     'neighbourhood_cleansed',
     #'neighbourhood_group_cleansed',
     'latitude',
     'longitude',
     #'property_type',
     'room_type',
     'accommodates',
     #'bathrooms' - unfortunately this is blank so bathrooms_text will need to be cleaned further,
     'bathrooms_text',
     'bedrooms',
     'beds',
     #'amenities',
     'price',
     'minimum_nights',
     'maximum_nights',
     #'minimum_minimum_nights',
     #'maximum_minimum_nights',
     #'minimum_maximum_nights',
     #'maximum_maximum_nights',
     #'minimum_nights_avg_ntm',
     #'maximum_nights_avg_ntm',
     #'calendar_updated',
     #'has_availability',  (could decide to keep availability in)
     #'availability_30',
     #'availability_60',
     #'availability_90',
     #'availability_365',
     #'calendar_last_scraped',
     #'number_of_reviews',
     #'number_of_reviews_ltm',
     'number_of_reviews_l30d',
     #'first_review',
     'last_review',
     'review_scores_rating',
     'review_scores_accuracy',
     'review_scores_cleanliness',
     'review_scores_checkin',
     'review_scores_communication',
     'review_scores_location',
     'review_scores_value',
     #'license',
     #'instant_bookable',
     #'calculated_host_listings_count',
     #'calculated_host_listings_count_entire_homes',
     #'calculated_host_listings_count_private_rooms',
     #'calculated_host_listings_count_shared_rooms',
     'reviews_per_month'
]]
pd.set_option('display.max_columns', None)
ARBNB_df2.head()

,listing_url,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_identity_verified,neighbourhood_cleansed,latitude,longitude,room_type,accommodates,bathrooms_text,bedrooms,beds,price,minimum_nights,maximum_nights,number_of_reviews_l30d,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month
0,https://www.airbnb.com/rooms/12899,within an hour,100%,100%,t,f,Concordia,45.56488,-122.63418,Entire home/apt,3,1 bath,2.0,2.0,$89.00,3,730,1,2021-11-30,4.93,4.94,4.98,4.99,4.99,4.93,4.94,4.08
1,https://www.airbnb.com/rooms/37676,within a day,100%,55%,t,t,Pearl,45.52564,-122.68273,Entire home/apt,3,1 bath,1.0,1.0,$140.00,30,730,1,2021-12-01,4.88,4.86,4.86,4.83,4.77,4.94,4.66,0.91
2,https://www.airbnb.com/rooms/41601,within an hour,100%,100%,t,t,Roseway,45.54804,-122.58541,Entire home/apt,2,1 bath,1.0,1.0,$129.00,3,29,0,2021-10-13,4.84,4.90,4.90,4.95,4.92,4.67,4.83,1.76
3,https://www.airbnb.com/rooms/45890,within an hour,100%,100%,t,t,Roseway,45.54793,-122.58358,Entire home/apt,2,1 bath,NaN,1.0,$133.00,3,29,1,2021-11-27,4.86,4.87,4.95,4.95,4.94,4.65,4.80,1.61
4,https://www.airbnb.com/rooms/61893,within an hour,100%,73%,t,t,Goose Hollow,45.52258,-122.69955,Entire home/apt,2,1 bath,1.0,1.0,$104.00,30,300,1,2021-12-01,5.00,5.00,5.00,4.93,5.00,5.00,4.93,0.24


In [185]:
#set new index - not needed with Katie's code on t and f
#ARBNB_df2_index=ARBNB_df2.set_index("id")

In [187]:
#filter to only look at entire home/apt
ARBNB_Portland_filter=ARBNB_df2.loc[(ARBNB_df2["room_type"]=="Entire home/apt")]
ARBNB_Portland_filter

,listing_url,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_identity_verified,neighbourhood_cleansed,latitude,longitude,room_type,accommodates,bathrooms_text,bedrooms,beds,price,minimum_nights,maximum_nights,number_of_reviews_l30d,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month
0,https://www.airbnb.com/rooms/12899,within an hour,100%,100%,t,f,Concordia,45.564880,-122.634180,Entire home/apt,3,1 bath,2.0,2.0,$89.00,3,730,1,2021-11-30,4.93,4.94,4.98,4.99,4.99,4.93,4.94,4.08
1,https://www.airbnb.com/rooms/37676,within a day,100%,55%,t,t,Pearl,45.525640,-122.682730,Entire home/apt,3,1 bath,1.0,1.0,$140.00,30,730,1,2021-12-01,4.88,4.86,4.86,4.83,4.77,4.94,4.66,0.91
2,https://www.airbnb.com/rooms/41601,within an hour,100%,100%,t,t,Roseway,45.548040,-122.585410,Entire home/apt,2,1 bath,1.0,1.0,$129.00,3,29,0,2021-10-13,4.84,4.90,4.90,4.95,4.92,4.67,4.83,1.76
3,https://www.airbnb.com/rooms/45890,within an hour,100%,100%,t,t,Roseway,45.547930,-122.583580,Entire home/apt,2,1 bath,NaN,1.0,$133.00,3,29,1,2021-11-27,4.86,4.87,4.95,4.95,4.94,4.65,4.80,1.61
4,https://www.airbnb.com/rooms/61893,within an hour,100%,73%,t,t,Goose Hollow,45.522580,-122.699550,Entire home/apt,2,1 bath,1.0,1.0,$104.00,30,300,1,2021-12-01,5.00,5.00,5.00,4.93,5.00,5.00,4.93,0.24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3729,https://www.airbnb.com/rooms/53829293,within an hour,100%,89%,f,t,Northwest District,45.531189,-122.690158,Entire home/apt,2,1 bath,NaN,1.0,$154.00,120,365,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3730,https://www.airbnb.com/rooms/53879278,within an hour,100%,89%,f,t,Hayhurst,45.484074,-122.747648,Entire home/apt,3,1 bath,1.0,1.0,$173.00,365,365,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3731,https://www.airbnb.com/rooms/53880025,within an hour,100%,93%,t,f,Northwest District,45.534796,-122.697880,Entire home/apt,4,2 baths,6.0,8.0,$234.00,1,14,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3732,https://www.airbnb.com/rooms/53891208,within an hour,89%,99%,t,t,King,45.561982,-122.661731,Entire home/apt,4,2.5 baths,2.0,2.0,$87.00,2,365,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [188]:
# convert t/f and true/false to boolean int
def change_tf(value):
    if value == "t":
        return 1
    else:
        return 0
ARBNB_Portland_filter['host_is_superhost'] = ARBNB_Portland_filter['host_is_superhost'].apply(change_tf)
ARBNB_Portland_filter['host_identity_verified'] = ARBNB_Portland_filter['host_identity_verified'].apply(change_tf)
ARBNB_Portland_filter.head()

C:\Users\laura\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\laura\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,listing_url,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_identity_verified,neighbourhood_cleansed,latitude,longitude,room_type,accommodates,bathrooms_text,bedrooms,beds,price,minimum_nights,maximum_nights,number_of_reviews_l30d,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month
0,https://www.airbnb.com/rooms/12899,within an hour,100%,100%,1,0,Concordia,45.56488,-122.63418,Entire home/apt,3,1 bath,2.0,2.0,$89.00,3,730,1,2021-11-30,4.93,4.94,4.98,4.99,4.99,4.93,4.94,4.08
1,https://www.airbnb.com/rooms/37676,within a day,100%,55%,1,1,Pearl,45.52564,-122.68273,Entire home/apt,3,1 bath,1.0,1.0,$140.00,30,730,1,2021-12-01,4.88,4.86,4.86,4.83,4.77,4.94,4.66,0.91
2,https://www.airbnb.com/rooms/41601,within an hour,100%,100%,1,1,Roseway,45.54804,-122.58541,Entire home/apt,2,1 bath,1.0,1.0,$129.00,3,29,0,2021-10-13,4.84,4.90,4.90,4.95,4.92,4.67,4.83,1.76
3,https://www.airbnb.com/rooms/45890,within an hour,100%,100%,1,1,Roseway,45.54793,-122.58358,Entire home/apt,2,1 bath,NaN,1.0,$133.00,3,29,1,2021-11-27,4.86,4.87,4.95,4.95,4.94,4.65,4.80,1.61
4,https://www.airbnb.com/rooms/61893,within an hour,100%,73%,1,1,Goose Hollow,45.52258,-122.69955,Entire home/apt,2,1 bath,1.0,1.0,$104.00,30,300,1,2021-12-01,5.00,5.00,5.00,4.93,5.00,5.00,4.93,0.24


In [189]:
#drop half-bath description since it is one data point and doesn't give a number
ARBNB_Portland_filter2=ARBNB_Portland_filter[ARBNB_Portland_filter.bathrooms_text != "Half-bath"]
ARBNB_Portland_filter2

,listing_url,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_identity_verified,neighbourhood_cleansed,latitude,longitude,room_type,accommodates,bathrooms_text,bedrooms,beds,price,minimum_nights,maximum_nights,number_of_reviews_l30d,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month
0,https://www.airbnb.com/rooms/12899,within an hour,100%,100%,1,0,Concordia,45.564880,-122.634180,Entire home/apt,3,1 bath,2.0,2.0,$89.00,3,730,1,2021-11-30,4.93,4.94,4.98,4.99,4.99,4.93,4.94,4.08
1,https://www.airbnb.com/rooms/37676,within a day,100%,55%,1,1,Pearl,45.525640,-122.682730,Entire home/apt,3,1 bath,1.0,1.0,$140.00,30,730,1,2021-12-01,4.88,4.86,4.86,4.83,4.77,4.94,4.66,0.91
2,https://www.airbnb.com/rooms/41601,within an hour,100%,100%,1,1,Roseway,45.548040,-122.585410,Entire home/apt,2,1 bath,1.0,1.0,$129.00,3,29,0,2021-10-13,4.84,4.90,4.90,4.95,4.92,4.67,4.83,1.76
3,https://www.airbnb.com/rooms/45890,within an hour,100%,100%,1,1,Roseway,45.547930,-122.583580,Entire home/apt,2,1 bath,NaN,1.0,$133.00,3,29,1,2021-11-27,4.86,4.87,4.95,4.95,4.94,4.65,4.80,1.61
4,https://www.airbnb.com/rooms/61893,within an hour,100%,73%,1,1,Goose Hollow,45.522580,-122.699550,Entire home/apt,2,1 bath,1.0,1.0,$104.00,30,300,1,2021-12-01,5.00,5.00,5.00,4.93,5.00,5.00,4.93,0.24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3729,https://www.airbnb.com/rooms/53829293,within an hour,100%,89%,0,1,Northwest District,45.531189,-122.690158,Entire home/apt,2,1 bath,NaN,1.0,$154.00,120,365,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3730,https://www.airbnb.com/rooms/53879278,within an hour,100%,89%,0,1,Hayhurst,45.484074,-122.747648,Entire home/apt,3,1 bath,1.0,1.0,$173.00,365,365,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3731,https://www.airbnb.com/rooms/53880025,within an hour,100%,93%,1,0,Northwest District,45.534796,-122.697880,Entire home/apt,4,2 baths,6.0,8.0,$234.00,1,14,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3732,https://www.airbnb.com/rooms/53891208,within an hour,89%,99%,1,1,King,45.561982,-122.661731,Entire home/apt,4,2.5 baths,2.0,2.0,$87.00,2,365,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [190]:
# Display an overview of the bathrooms_text column
ARBNB_Portland_filter2['bathrooms_text'].value_counts()

1 bath       2373
2 baths       332
1.5 baths     141
2.5 baths      91
3 baths        55
3.5 baths      17
4 baths         7
5 baths         2
6.5 baths       1
Name: bathrooms_text, dtype: int64

In [191]:
# Clean up bathrooms_text category. 
ARBNB_Portland_filter2['bathrooms_text'] = ARBNB_Portland_filter2['bathrooms_text'].replace(
    {'1 bath': 1, '1.5 baths': 1.5, '2 baths': 2, "2.5 baths": 2.5, "3 baths": 3,
     "3.5 baths":3.5, "4 baths":4, "5 baths": 5, "6.5 baths": 6.5 })
ARBNB_Portland_filter2

C:\Users\laura\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,listing_url,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_identity_verified,neighbourhood_cleansed,latitude,longitude,room_type,accommodates,bathrooms_text,bedrooms,beds,price,minimum_nights,maximum_nights,number_of_reviews_l30d,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month
0,https://www.airbnb.com/rooms/12899,within an hour,100%,100%,1,0,Concordia,45.564880,-122.634180,Entire home/apt,3,1.0,2.0,2.0,$89.00,3,730,1,2021-11-30,4.93,4.94,4.98,4.99,4.99,4.93,4.94,4.08
1,https://www.airbnb.com/rooms/37676,within a day,100%,55%,1,1,Pearl,45.525640,-122.682730,Entire home/apt,3,1.0,1.0,1.0,$140.00,30,730,1,2021-12-01,4.88,4.86,4.86,4.83,4.77,4.94,4.66,0.91
2,https://www.airbnb.com/rooms/41601,within an hour,100%,100%,1,1,Roseway,45.548040,-122.585410,Entire home/apt,2,1.0,1.0,1.0,$129.00,3,29,0,2021-10-13,4.84,4.90,4.90,4.95,4.92,4.67,4.83,1.76
3,https://www.airbnb.com/rooms/45890,within an hour,100%,100%,1,1,Roseway,45.547930,-122.583580,Entire home/apt,2,1.0,NaN,1.0,$133.00,3,29,1,2021-11-27,4.86,4.87,4.95,4.95,4.94,4.65,4.80,1.61
4,https://www.airbnb.com/rooms/61893,within an hour,100%,73%,1,1,Goose Hollow,45.522580,-122.699550,Entire home/apt,2,1.0,1.0,1.0,$104.00,30,300,1,2021-12-01,5.00,5.00,5.00,4.93,5.00,5.00,4.93,0.24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3729,https://www.airbnb.com/rooms/53829293,within an hour,100%,89%,0,1,Northwest District,45.531189,-122.690158,Entire home/apt,2,1.0,NaN,1.0,$154.00,120,365,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3730,https://www.airbnb.com/rooms/53879278,within an hour,100%,89%,0,1,Hayhurst,45.484074,-122.747648,Entire home/apt,3,1.0,1.0,1.0,$173.00,365,365,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3731,https://www.airbnb.com/rooms/53880025,within an hour,100%,93%,1,0,Northwest District,45.534796,-122.697880,Entire home/apt,4,2.0,6.0,8.0,$234.00,1,14,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3732,https://www.airbnb.com/rooms/53891208,within an hour,89%,99%,1,1,King,45.561982,-122.661731,Entire home/apt,4,2.5,2.0,2.0,$87.00,2,365,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [192]:
# Clean up bathrooms_text category using iloc
ARBNB_Portland_filter2['bathrooms_text'] = ARBNB_Portland_filter2['bathrooms_text'].replace(
    {'1 bath': 1, '1.5 baths': 1.5, '2 baths': 2, "2.5 baths": 2.5, "3 baths": 3,
     "3.5 baths":3.5, "4 baths":4, "5 baths": 5, "6.5 baths": 6.5 })
ARBNB_Portland_filter2

C:\Users\laura\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,listing_url,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_identity_verified,neighbourhood_cleansed,latitude,longitude,room_type,accommodates,bathrooms_text,bedrooms,beds,price,minimum_nights,maximum_nights,number_of_reviews_l30d,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month
0,https://www.airbnb.com/rooms/12899,within an hour,100%,100%,1,0,Concordia,45.564880,-122.634180,Entire home/apt,3,1.0,2.0,2.0,$89.00,3,730,1,2021-11-30,4.93,4.94,4.98,4.99,4.99,4.93,4.94,4.08
1,https://www.airbnb.com/rooms/37676,within a day,100%,55%,1,1,Pearl,45.525640,-122.682730,Entire home/apt,3,1.0,1.0,1.0,$140.00,30,730,1,2021-12-01,4.88,4.86,4.86,4.83,4.77,4.94,4.66,0.91
2,https://www.airbnb.com/rooms/41601,within an hour,100%,100%,1,1,Roseway,45.548040,-122.585410,Entire home/apt,2,1.0,1.0,1.0,$129.00,3,29,0,2021-10-13,4.84,4.90,4.90,4.95,4.92,4.67,4.83,1.76
3,https://www.airbnb.com/rooms/45890,within an hour,100%,100%,1,1,Roseway,45.547930,-122.583580,Entire home/apt,2,1.0,NaN,1.0,$133.00,3,29,1,2021-11-27,4.86,4.87,4.95,4.95,4.94,4.65,4.80,1.61
4,https://www.airbnb.com/rooms/61893,within an hour,100%,73%,1,1,Goose Hollow,45.522580,-122.699550,Entire home/apt,2,1.0,1.0,1.0,$104.00,30,300,1,2021-12-01,5.00,5.00,5.00,4.93,5.00,5.00,4.93,0.24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3729,https://www.airbnb.com/rooms/53829293,within an hour,100%,89%,0,1,Northwest District,45.531189,-122.690158,Entire home/apt,2,1.0,NaN,1.0,$154.00,120,365,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3730,https://www.airbnb.com/rooms/53879278,within an hour,100%,89%,0,1,Hayhurst,45.484074,-122.747648,Entire home/apt,3,1.0,1.0,1.0,$173.00,365,365,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3731,https://www.airbnb.com/rooms/53880025,within an hour,100%,93%,1,0,Northwest District,45.534796,-122.697880,Entire home/apt,4,2.0,6.0,8.0,$234.00,1,14,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3732,https://www.airbnb.com/rooms/53891208,within an hour,89%,99%,1,1,King,45.561982,-122.661731,Entire home/apt,4,2.5,2.0,2.0,$87.00,2,365,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [193]:
ARBNB_Portland_filter2['bathrooms_text'].value_counts()

1.0    2373
2.0     332
1.5     141
2.5      91
3.0      55
3.5      17
4.0       7
5.0       2
6.5       1
Name: bathrooms_text, dtype: int64

In [194]:
# changes bathrooms_text to bathrooms
ARBNB_Portland_filter2.rename(columns = {'bathrooms_text':'bathrooms'}, inplace = True)
ARBNB_Portland_filter2

C:\Users\laura\anaconda3\envs\PythonData\lib\site-packages\pandas\core\frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,listing_url,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_identity_verified,neighbourhood_cleansed,latitude,longitude,room_type,accommodates,bathrooms,bedrooms,beds,price,minimum_nights,maximum_nights,number_of_reviews_l30d,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month
0,https://www.airbnb.com/rooms/12899,within an hour,100%,100%,1,0,Concordia,45.564880,-122.634180,Entire home/apt,3,1.0,2.0,2.0,$89.00,3,730,1,2021-11-30,4.93,4.94,4.98,4.99,4.99,4.93,4.94,4.08
1,https://www.airbnb.com/rooms/37676,within a day,100%,55%,1,1,Pearl,45.525640,-122.682730,Entire home/apt,3,1.0,1.0,1.0,$140.00,30,730,1,2021-12-01,4.88,4.86,4.86,4.83,4.77,4.94,4.66,0.91
2,https://www.airbnb.com/rooms/41601,within an hour,100%,100%,1,1,Roseway,45.548040,-122.585410,Entire home/apt,2,1.0,1.0,1.0,$129.00,3,29,0,2021-10-13,4.84,4.90,4.90,4.95,4.92,4.67,4.83,1.76
3,https://www.airbnb.com/rooms/45890,within an hour,100%,100%,1,1,Roseway,45.547930,-122.583580,Entire home/apt,2,1.0,NaN,1.0,$133.00,3,29,1,2021-11-27,4.86,4.87,4.95,4.95,4.94,4.65,4.80,1.61
4,https://www.airbnb.com/rooms/61893,within an hour,100%,73%,1,1,Goose Hollow,45.522580,-122.699550,Entire home/apt,2,1.0,1.0,1.0,$104.00,30,300,1,2021-12-01,5.00,5.00,5.00,4.93,5.00,5.00,4.93,0.24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3729,https://www.airbnb.com/rooms/53829293,within an hour,100%,89%,0,1,Northwest District,45.531189,-122.690158,Entire home/apt,2,1.0,NaN,1.0,$154.00,120,365,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3730,https://www.airbnb.com/rooms/53879278,within an hour,100%,89%,0,1,Hayhurst,45.484074,-122.747648,Entire home/apt,3,1.0,1.0,1.0,$173.00,365,365,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3731,https://www.airbnb.com/rooms/53880025,within an hour,100%,93%,1,0,Northwest District,45.534796,-122.697880,Entire home/apt,4,2.0,6.0,8.0,$234.00,1,14,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3732,https://www.airbnb.com/rooms/53891208,within an hour,89%,99%,1,1,King,45.561982,-122.661731,Entire home/apt,4,2.5,2.0,2.0,$87.00,2,365,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [195]:
ARBNB_Portland_filter2.isna().sum()

listing_url                      0
host_response_time             316
host_response_rate             316
host_acceptance_rate           192
host_is_superhost                0
host_identity_verified           0
neighbourhood_cleansed           0
latitude                         0
longitude                        0
room_type                        0
accommodates                     0
bathrooms                        0
bedrooms                       409
beds                            35
price                            0
minimum_nights                   0
maximum_nights                   0
number_of_reviews_l30d           0
last_review                    258
review_scores_rating           258
review_scores_accuracy         261
review_scores_cleanliness      261
review_scores_checkin          261
review_scores_communication    261
review_scores_location         261
review_scores_value            261
reviews_per_month              258
dtype: int64

In [196]:
ARBNB_Portland_dropna=ARBNB_Portland_filter2.dropna()
ARBNB_Portland_dropna.describe()

,host_is_superhost,host_identity_verified,latitude,longitude,accommodates,bathrooms,bedrooms,beds,minimum_nights,maximum_nights,number_of_reviews_l30d,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month
count,2101.00000,2101.00000,2101.000000,2101.000000,2101.000000,2101.000000,2101.000000,2101.000000,2101.000000,2101.000000,2101.000000,2101.000000,2101.000000,2101.000000,2101.000000,2101.000000,2101.000000,2101.000000,2101.000000
mean,0.68634,0.84436,45.529788,-122.651957,4.013803,1.257258,1.646835,2.112327,7.279867,531.967634,1.907663,4.852432,4.886069,4.859710,4.921266,4.915988,4.881447,4.795940,2.738696
std,0.46409,0.36260,0.032884,0.043276,2.033599,0.539489,0.962623,1.304478,11.577759,517.294375,2.232881,0.285615,0.276352,0.282657,0.274738,0.271439,0.234751,0.306419,2.267940
min,0.00000,0.00000,45.436120,-122.832860,1.000000,1.000000,1.000000,1.000000,1.000000,3.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.010000
25%,0.00000,1.00000,45.508980,-122.680300,2.000000,1.000000,1.000000,1.000000,1.000000,30.000000,0.000000,4.830000,4.880000,4.840000,4.940000,4.920000,4.860000,4.750000,0.920000
50%,1.00000,1.00000,45.528520,-122.651370,4.000000,1.000000,1.000000,2.000000,2.000000,360.000000,1.000000,4.920000,4.950000,4.930000,4.980000,4.980000,4.930000,4.860000,2.240000
75%,1.00000,1.00000,45.556690,-122.626320,5.000000,1.000000,2.000000,2.000000,4.000000,1125.000000,3.000000,4.980000,5.000000,4.990000,5.000000,5.000000,4.990000,4.930000,3.930000
max,1.00000,1.00000,45.612050,-122.496730,16.000000,5.000000,9.000000,16.000000,120.000000,1125.000000,14.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,14.690000


In [197]:
# convert dates from object to date
ARBNB_Portland_dropna["last_review"]=ARBNB_Portland_dropna["last_review"].apply(pd.to_datetime)
ARBNB_Portland_dropna.head()

C:\Users\laura\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,listing_url,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_identity_verified,neighbourhood_cleansed,latitude,longitude,room_type,accommodates,bathrooms,bedrooms,beds,price,minimum_nights,maximum_nights,number_of_reviews_l30d,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month
0,https://www.airbnb.com/rooms/12899,within an hour,100%,100%,1,0,Concordia,45.56488,-122.63418,Entire home/apt,3,1.0,2.0,2.0,$89.00,3,730,1,2021-11-30,4.93,4.94,4.98,4.99,4.99,4.93,4.94,4.08
1,https://www.airbnb.com/rooms/37676,within a day,100%,55%,1,1,Pearl,45.52564,-122.68273,Entire home/apt,3,1.0,1.0,1.0,$140.00,30,730,1,2021-12-01,4.88,4.86,4.86,4.83,4.77,4.94,4.66,0.91
2,https://www.airbnb.com/rooms/41601,within an hour,100%,100%,1,1,Roseway,45.54804,-122.58541,Entire home/apt,2,1.0,1.0,1.0,$129.00,3,29,0,2021-10-13,4.84,4.90,4.90,4.95,4.92,4.67,4.83,1.76
4,https://www.airbnb.com/rooms/61893,within an hour,100%,73%,1,1,Goose Hollow,45.52258,-122.69955,Entire home/apt,2,1.0,1.0,1.0,$104.00,30,300,1,2021-12-01,5.00,5.00,5.00,4.93,5.00,5.00,4.93,0.24
6,https://www.airbnb.com/rooms/67036,within a day,100%,0%,0,1,Sullivan's Gulch,45.53103,-122.64448,Entire home/apt,12,2.0,4.0,5.0,$425.00,4,365,0,2020-02-27,4.93,5.00,5.00,5.00,5.00,4.79,4.89,0.23


In [198]:
ARBNB_Portland_dropna.isna().sum()

listing_url                    0
host_response_time             0
host_response_rate             0
host_acceptance_rate           0
host_is_superhost              0
host_identity_verified         0
neighbourhood_cleansed         0
latitude                       0
longitude                      0
room_type                      0
accommodates                   0
bathrooms                      0
bedrooms                       0
beds                           0
price                          0
minimum_nights                 0
maximum_nights                 0
number_of_reviews_l30d         0
last_review                    0
review_scores_rating           0
review_scores_accuracy         0
review_scores_cleanliness      0
review_scores_checkin          0
review_scores_communication    0
review_scores_location         0
review_scores_value            0
reviews_per_month              0
dtype: int64

In [199]:
ARBNB_Portland_dropna.dtypes

listing_url                            object
host_response_time                     object
host_response_rate                     object
host_acceptance_rate                   object
host_is_superhost                       int64
host_identity_verified                  int64
neighbourhood_cleansed                 object
latitude                              float64
longitude                             float64
room_type                              object
accommodates                            int64
bathrooms                             float64
bedrooms                              float64
beds                                  float64
price                                  object
minimum_nights                          int64
maximum_nights                          int64
number_of_reviews_l30d                  int64
last_review                    datetime64[ns]
review_scores_rating                  float64
review_scores_accuracy                float64
review_scores_cleanliness         

In [200]:
## no longer needed used Katie's code
# #change host_is_superhost to boolean and host_identity_verified to boolean
# d={"t":True,"f":False}
# ARBNB_Portland_b1=ARBNB_Portland_dropna['host_is_superhost'].map(d)
# ARBNB_Portland_b1

In [201]:
## no longer needed used Katie's code
# #change host_identity_verified to boolean
# ARBNB_Portland_b2=ARBNB_Portland_dropna['host_identity_verified'].map(d)

In [202]:
## no longer needed used Katie's code
# join_df=pd.concat([ARBNB_Portland_dropna, ARBNB_Portland_b1],axis=1)
# join_df

In [203]:
## no longer needed used Katie's code
# join_df.columns = [*join_df.columns[:-1], 'is_superhost']
# join_df

In [204]:
## no longer needed used Katie's code
# join_df2=pd.concat([join_df, ARBNB_Portland_b2],axis=1)
# join_df2

In [205]:
## no longer needed used Katie's code
# join_df2.columns = [*join_df2.columns[:-1], 'is_verified']
# join_df2

In [206]:
## no longer needed used Katie's code
# join_df2.drop(columns=["host_is_superhost", "host_identity_verified"])

In [207]:
#strip price of $ commma and .00 from price
ARBNB_Portland_dropna["price"]=ARBNB_Portland_dropna["price"].str.strip("$")
ARBNB_Portland_dropna["price"]=ARBNB_Portland_dropna["price"].str.replace(",","")
ARBNB_Portland_dropna["price"]=ARBNB_Portland_dropna["price"].str.strip(".00")
ARBNB_Portland_dropna

C:\Users\laura\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\laura\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\laura\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFr

,listing_url,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_identity_verified,neighbourhood_cleansed,latitude,longitude,room_type,accommodates,bathrooms,bedrooms,beds,price,minimum_nights,maximum_nights,number_of_reviews_l30d,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month
0,https://www.airbnb.com/rooms/12899,within an hour,100%,100%,1,0,Concordia,45.564880,-122.634180,Entire home/apt,3,1.0,2.0,2.0,89,3,730,1,2021-11-30,4.93,4.94,4.98,4.99,4.99,4.93,4.94,4.08
1,https://www.airbnb.com/rooms/37676,within a day,100%,55%,1,1,Pearl,45.525640,-122.682730,Entire home/apt,3,1.0,1.0,1.0,14,30,730,1,2021-12-01,4.88,4.86,4.86,4.83,4.77,4.94,4.66,0.91
2,https://www.airbnb.com/rooms/41601,within an hour,100%,100%,1,1,Roseway,45.548040,-122.585410,Entire home/apt,2,1.0,1.0,1.0,129,3,29,0,2021-10-13,4.84,4.90,4.90,4.95,4.92,4.67,4.83,1.76
4,https://www.airbnb.com/rooms/61893,within an hour,100%,73%,1,1,Goose Hollow,45.522580,-122.699550,Entire home/apt,2,1.0,1.0,1.0,104,30,300,1,2021-12-01,5.00,5.00,5.00,4.93,5.00,5.00,4.93,0.24
6,https://www.airbnb.com/rooms/67036,within a day,100%,0%,0,1,Sullivan's Gulch,45.531030,-122.644480,Entire home/apt,12,2.0,4.0,5.0,425,4,365,0,2020-02-27,4.93,5.00,5.00,5.00,5.00,4.79,4.89,0.23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3686,https://www.airbnb.com/rooms/53623129,within an hour,100%,100%,0,1,Pearl,45.529118,-122.680400,Entire home/apt,3,1.0,1.0,2.0,233,1,365,1,2021-12-04,4.00,5.00,5.00,2.00,5.00,5.00,5.00,1.00
3690,https://www.airbnb.com/rooms/53631855,within an hour,100%,86%,0,1,Maywood Park,45.553293,-122.561127,Entire home/apt,4,1.0,1.0,1.0,64,1,365,1,2021-12-10,5.00,5.00,5.00,5.00,5.00,5.00,5.00,1.00
3691,https://www.airbnb.com/rooms/53644475,within an hour,100%,100%,0,1,Southwest Hills,45.494583,-122.707946,Entire home/apt,2,1.0,1.0,1.0,137,3,29,1,2021-12-13,5.00,5.00,5.00,5.00,5.00,5.00,5.00,1.00
3692,https://www.airbnb.com/rooms/53649962,within an hour,100%,100%,0,1,Humboldt,45.555253,-122.666735,Entire home/apt,7,2.5,3.0,3.0,203,2,1125,1,2021-12-12,5.00,5.00,5.00,5.00,5.00,5.00,5.00,1.00


In [208]:
#strip host_response_rate of %
ARBNB_Portland_dropna["host_response_rate"]=ARBNB_Portland_dropna["host_response_rate"].str.replace("%","")
ARBNB_Portland_dropna

C:\Users\laura\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,listing_url,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_identity_verified,neighbourhood_cleansed,latitude,longitude,room_type,accommodates,bathrooms,bedrooms,beds,price,minimum_nights,maximum_nights,number_of_reviews_l30d,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month
0,https://www.airbnb.com/rooms/12899,within an hour,100,100%,1,0,Concordia,45.564880,-122.634180,Entire home/apt,3,1.0,2.0,2.0,89,3,730,1,2021-11-30,4.93,4.94,4.98,4.99,4.99,4.93,4.94,4.08
1,https://www.airbnb.com/rooms/37676,within a day,100,55%,1,1,Pearl,45.525640,-122.682730,Entire home/apt,3,1.0,1.0,1.0,14,30,730,1,2021-12-01,4.88,4.86,4.86,4.83,4.77,4.94,4.66,0.91
2,https://www.airbnb.com/rooms/41601,within an hour,100,100%,1,1,Roseway,45.548040,-122.585410,Entire home/apt,2,1.0,1.0,1.0,129,3,29,0,2021-10-13,4.84,4.90,4.90,4.95,4.92,4.67,4.83,1.76
4,https://www.airbnb.com/rooms/61893,within an hour,100,73%,1,1,Goose Hollow,45.522580,-122.699550,Entire home/apt,2,1.0,1.0,1.0,104,30,300,1,2021-12-01,5.00,5.00,5.00,4.93,5.00,5.00,4.93,0.24
6,https://www.airbnb.com/rooms/67036,within a day,100,0%,0,1,Sullivan's Gulch,45.531030,-122.644480,Entire home/apt,12,2.0,4.0,5.0,425,4,365,0,2020-02-27,4.93,5.00,5.00,5.00,5.00,4.79,4.89,0.23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3686,https://www.airbnb.com/rooms/53623129,within an hour,100,100%,0,1,Pearl,45.529118,-122.680400,Entire home/apt,3,1.0,1.0,2.0,233,1,365,1,2021-12-04,4.00,5.00,5.00,2.00,5.00,5.00,5.00,1.00
3690,https://www.airbnb.com/rooms/53631855,within an hour,100,86%,0,1,Maywood Park,45.553293,-122.561127,Entire home/apt,4,1.0,1.0,1.0,64,1,365,1,2021-12-10,5.00,5.00,5.00,5.00,5.00,5.00,5.00,1.00
3691,https://www.airbnb.com/rooms/53644475,within an hour,100,100%,0,1,Southwest Hills,45.494583,-122.707946,Entire home/apt,2,1.0,1.0,1.0,137,3,29,1,2021-12-13,5.00,5.00,5.00,5.00,5.00,5.00,5.00,1.00
3692,https://www.airbnb.com/rooms/53649962,within an hour,100,100%,0,1,Humboldt,45.555253,-122.666735,Entire home/apt,7,2.5,3.0,3.0,203,2,1125,1,2021-12-12,5.00,5.00,5.00,5.00,5.00,5.00,5.00,1.00


In [209]:
#strip host_acceptance_rate of %
ARBNB_Portland_dropna["host_acceptance_rate"]=ARBNB_Portland_dropna["host_acceptance_rate"].str.replace("%","")
ARBNB_Portland_dropna

C:\Users\laura\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,listing_url,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_identity_verified,neighbourhood_cleansed,latitude,longitude,room_type,accommodates,bathrooms,bedrooms,beds,price,minimum_nights,maximum_nights,number_of_reviews_l30d,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month
0,https://www.airbnb.com/rooms/12899,within an hour,100,100,1,0,Concordia,45.564880,-122.634180,Entire home/apt,3,1.0,2.0,2.0,89,3,730,1,2021-11-30,4.93,4.94,4.98,4.99,4.99,4.93,4.94,4.08
1,https://www.airbnb.com/rooms/37676,within a day,100,55,1,1,Pearl,45.525640,-122.682730,Entire home/apt,3,1.0,1.0,1.0,14,30,730,1,2021-12-01,4.88,4.86,4.86,4.83,4.77,4.94,4.66,0.91
2,https://www.airbnb.com/rooms/41601,within an hour,100,100,1,1,Roseway,45.548040,-122.585410,Entire home/apt,2,1.0,1.0,1.0,129,3,29,0,2021-10-13,4.84,4.90,4.90,4.95,4.92,4.67,4.83,1.76
4,https://www.airbnb.com/rooms/61893,within an hour,100,73,1,1,Goose Hollow,45.522580,-122.699550,Entire home/apt,2,1.0,1.0,1.0,104,30,300,1,2021-12-01,5.00,5.00,5.00,4.93,5.00,5.00,4.93,0.24
6,https://www.airbnb.com/rooms/67036,within a day,100,0,0,1,Sullivan's Gulch,45.531030,-122.644480,Entire home/apt,12,2.0,4.0,5.0,425,4,365,0,2020-02-27,4.93,5.00,5.00,5.00,5.00,4.79,4.89,0.23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3686,https://www.airbnb.com/rooms/53623129,within an hour,100,100,0,1,Pearl,45.529118,-122.680400,Entire home/apt,3,1.0,1.0,2.0,233,1,365,1,2021-12-04,4.00,5.00,5.00,2.00,5.00,5.00,5.00,1.00
3690,https://www.airbnb.com/rooms/53631855,within an hour,100,86,0,1,Maywood Park,45.553293,-122.561127,Entire home/apt,4,1.0,1.0,1.0,64,1,365,1,2021-12-10,5.00,5.00,5.00,5.00,5.00,5.00,5.00,1.00
3691,https://www.airbnb.com/rooms/53644475,within an hour,100,100,0,1,Southwest Hills,45.494583,-122.707946,Entire home/apt,2,1.0,1.0,1.0,137,3,29,1,2021-12-13,5.00,5.00,5.00,5.00,5.00,5.00,5.00,1.00
3692,https://www.airbnb.com/rooms/53649962,within an hour,100,100,0,1,Humboldt,45.555253,-122.666735,Entire home/apt,7,2.5,3.0,3.0,203,2,1125,1,2021-12-12,5.00,5.00,5.00,5.00,5.00,5.00,5.00,1.00


In [210]:
#convert to float
ARBNB_Portland_dropna["host_response_rate"]=ARBNB_Portland_dropna["host_response_rate"].astype(float)
ARBNB_Portland_dropna["host_acceptance_rate"]=ARBNB_Portland_dropna["host_acceptance_rate"].astype(float)
ARBNB_Portland_dropna["price"]=ARBNB_Portland_dropna["price"].astype(float)

C:\Users\laura\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\laura\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\laura\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFr

In [211]:
ARBNB_Portland_dropna.drop(columns=["room_type"])

,listing_url,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_identity_verified,neighbourhood_cleansed,latitude,longitude,accommodates,bathrooms,bedrooms,beds,price,minimum_nights,maximum_nights,number_of_reviews_l30d,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month
0,https://www.airbnb.com/rooms/12899,within an hour,100.0,100.0,1,0,Concordia,45.564880,-122.634180,3,1.0,2.0,2.0,89.0,3,730,1,2021-11-30,4.93,4.94,4.98,4.99,4.99,4.93,4.94,4.08
1,https://www.airbnb.com/rooms/37676,within a day,100.0,55.0,1,1,Pearl,45.525640,-122.682730,3,1.0,1.0,1.0,14.0,30,730,1,2021-12-01,4.88,4.86,4.86,4.83,4.77,4.94,4.66,0.91
2,https://www.airbnb.com/rooms/41601,within an hour,100.0,100.0,1,1,Roseway,45.548040,-122.585410,2,1.0,1.0,1.0,129.0,3,29,0,2021-10-13,4.84,4.90,4.90,4.95,4.92,4.67,4.83,1.76
4,https://www.airbnb.com/rooms/61893,within an hour,100.0,73.0,1,1,Goose Hollow,45.522580,-122.699550,2,1.0,1.0,1.0,104.0,30,300,1,2021-12-01,5.00,5.00,5.00,4.93,5.00,5.00,4.93,0.24
6,https://www.airbnb.com/rooms/67036,within a day,100.0,0.0,0,1,Sullivan's Gulch,45.531030,-122.644480,12,2.0,4.0,5.0,425.0,4,365,0,2020-02-27,4.93,5.00,5.00,5.00,5.00,4.79,4.89,0.23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3686,https://www.airbnb.com/rooms/53623129,within an hour,100.0,100.0,0,1,Pearl,45.529118,-122.680400,3,1.0,1.0,2.0,233.0,1,365,1,2021-12-04,4.00,5.00,5.00,2.00,5.00,5.00,5.00,1.00
3690,https://www.airbnb.com/rooms/53631855,within an hour,100.0,86.0,0,1,Maywood Park,45.553293,-122.561127,4,1.0,1.0,1.0,64.0,1,365,1,2021-12-10,5.00,5.00,5.00,5.00,5.00,5.00,5.00,1.00
3691,https://www.airbnb.com/rooms/53644475,within an hour,100.0,100.0,0,1,Southwest Hills,45.494583,-122.707946,2,1.0,1.0,1.0,137.0,3,29,1,2021-12-13,5.00,5.00,5.00,5.00,5.00,5.00,5.00,1.00
3692,https://www.airbnb.com/rooms/53649962,within an hour,100.0,100.0,0,1,Humboldt,45.555253,-122.666735,7,2.5,3.0,3.0,203.0,2,1125,1,2021-12-12,5.00,5.00,5.00,5.00,5.00,5.00,5.00,1.00


In [212]:
ARBNB_Portland_dropna.dtypes

listing_url                            object
host_response_time                     object
host_response_rate                    float64
host_acceptance_rate                  float64
host_is_superhost                       int64
host_identity_verified                  int64
neighbourhood_cleansed                 object
latitude                              float64
longitude                             float64
room_type                              object
accommodates                            int64
bathrooms                             float64
bedrooms                              float64
beds                                  float64
price                                 float64
minimum_nights                          int64
maximum_nights                          int64
number_of_reviews_l30d                  int64
last_review                    datetime64[ns]
review_scores_rating                  float64
review_scores_accuracy                float64
review_scores_cleanliness         

In [213]:
ARBNB_Portland_dropna.to_csv("Resources/cleaned_ARBNB.csv")

# ZILLOW DATA

In [53]:
# Store filepath in a variable
ZLLW = "Resources/portland_housing.csv"

In [54]:
# Read our Data file with the pandas library
# Not every CSV requires an encoding, but be aware this can come up
ZLLW_df = pd.read_csv(ZLLW, encoding="ISO-8859-1")

C:\Users\laura\anaconda3\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (20,50,161,200) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [55]:
# Show just the header
ZLLW_df.head()

,ï»¿abbreviatedAddress,address/city,address/zipcode,bathrooms,bedrooms,brokerageName,dateSold,daysOnZillow,description,favoriteCount,homeStatus,homeType,lastSoldPrice,latitude,livingArea,longitude,lotSize,pageViewCount,postingContact/name,price,priceHistory/0/attributeSource/infoString1,priceHistory/0/attributeSource/infoString2,priceHistory/0/attributeSource/infoString3,priceHistory/0/buyerAgent/name,priceHistory/0/buyerAgent/photo/url,priceHistory/0/buyerAgent/profileUrl,priceHistory/0/date,priceHistory/0/event,priceHistory/0/postingIsRental,priceHistory/0/price,priceHistory/0/priceChangeRate,priceHistory/0/pricePerSquareFoot,priceHistory/0/sellerAgent/name,priceHistory/0/sellerAgent/photo/url,priceHistory/0/sellerAgent/profileUrl,priceHistory/0/showCountyLink,priceHistory/0/source,priceHistory/0/time,priceHistory/1/attributeSource/infoString1,priceHistory/1/attributeSource/infoString2,priceHistory/1/attributeSource/infoString3,priceHistory/1/date,priceHistory/1/event,priceHistory/1/postingIsRental,priceHistory/1/price,priceHistory/1/priceChangeRate,priceHistory/1/pricePerSquareFoot,priceHistory/1/showCountyLink,priceHistory/1/source,priceHistory/1/time,priceHistory/2/attributeSource/infoString1,priceHistory/2/attributeSource/infoString2,priceHistory/2/attributeSource/infoString3,priceHistory/2/date,priceHistory/2/event,priceHistory/2/postingIsRental,priceHistory/2/price,priceHistory/2/priceChangeRate,priceHistory/2/pricePerSquareFoot,priceHistory/2/showCountyLink,priceHistory/2/source,priceHistory/2/time,priceHistory/3/attributeSource/infoString2,priceHistory/3/date,priceHistory/3/event,priceHistory/3/postingIsRental,priceHistory/3/price,priceHistory/3/priceChangeRate,priceHistory/3/pricePerSquareFoot,priceHistory/3/showCountyLink,priceHistory/3/source,priceHistory/3/time,priceHistory/4/attributeSource/infoString2,priceHistory/4/date,priceHistory/4/event,priceHistory/4/postingIsRental,priceHistory/4/price,priceHistory/4/priceChangeRate,priceHistory/4/pricePerSquareFoot,priceHistory/4/showCountyLink,priceHistory/4/source,priceHistory/4/time,priceHistory/5/attributeSource/infoString2,priceHistory/5/date,priceHistory/5/event,priceHistory/5/postingIsRental,priceHistory/5/price,priceHistory/5/priceChangeRate,priceHistory/5/pricePerSquareFoot,priceHistory/5/showCountyLink,priceHistory/5/source,priceHistory/5/time,priceHistory/6/attributeSource/infoString2,priceHistory/6/date,priceHistory/6/event,priceHistory/6/postingIsRental,priceHistory/6/price,priceHistory/6/priceChangeRate,priceHistory/6/pricePerSquareFoot,priceHistory/6/showCountyLink,priceHistory/6/source,priceHistory/6/time,priceHistory/7/attributeSource/infoString2,priceHistory/7/date,priceHistory/7/event,priceHistory/7/postingIsRental,priceHistory/7/price,priceHistory/7/priceChangeRate,priceHistory/7/pricePerSquareFoot,priceHistory/7/showCountyLink,priceHistory/7/source,priceHistory/7/time,priceHistory/8/attributeSource/infoString2,priceHistory/8/date,priceHistory/8/event,priceHistory/8/postingIsRental,priceHistory/8/price,priceHistory/8/priceChangeRate,priceHistory/8/pricePerSquareFoot,priceHistory/8/showCountyLink,priceHistory/8/source,priceHistory/8/time,priceHistory/9/attributeSource/infoString2,priceHistory/9/date,priceHistory/9/event,priceHistory/9/postingIsRental,priceHistory/9/price,priceHistory/9/priceChangeRate,priceHistory/9/pricePerSquareFoot,priceHistory/9/showCountyLink,priceHistory/9/source,priceHistory/9/time,priceHistory/10/attributeSource/infoString2,priceHistory/10/date,priceHistory/10/event,priceHistory/10/postingIsRental,priceHistory/10/price,priceHistory/10/priceChangeRate,priceHistory/10/showCountyLink,priceHistory/10/source,priceHistory/10/time,propertyTaxRate,rentZestimate,rentalApplicationsAcceptedType,resoFacts/accessibilityFeatures/0,resoFacts/accessibilityFeatures/1,resoFacts/appliances/0,resoFacts/appliances/1,resoFacts/appliances/2,resoFacts/appliances/3,resoFacts/appliances/4,resoFacts/appliances/5,resoFacts/appliances/6,resoFacts/applianc

In [56]:
column_list2 = ZLLW_df.columns.values.tolist()

column_list2

['ï»¿abbreviatedAddress',
 'address/city',
 'address/zipcode',
 'bathrooms',
 'bedrooms',
 'brokerageName',
 'dateSold',
 'daysOnZillow',
 'description',
 'favoriteCount',
 'homeStatus',
 'homeType',
 'lastSoldPrice',
 'latitude',
 'livingArea',
 'longitude',
 'lotSize',
 'pageViewCount',
 'postingContact/name',
 'price',
 'priceHistory/0/attributeSource/infoString1',
 'priceHistory/0/attributeSource/infoString2',
 'priceHistory/0/attributeSource/infoString3',
 'priceHistory/0/buyerAgent/name',
 'priceHistory/0/buyerAgent/photo/url',
 'priceHistory/0/buyerAgent/profileUrl',
 'priceHistory/0/date',
 'priceHistory/0/event',
 'priceHistory/0/postingIsRental',
 'priceHistory/0/price',
 'priceHistory/0/priceChangeRate',
 'priceHistory/0/pricePerSquareFoot',
 'priceHistory/0/sellerAgent/name',
 'priceHistory/0/sellerAgent/photo/url',
 'priceHistory/0/sellerAgent/profileUrl',
 'priceHistory/0/showCountyLink',
 'priceHistory/0/source',
 'priceHistory/0/time',
 'priceHistory/1/attributeSource/i

In [57]:
ZLLW_df.dtypes

ï»¿abbreviatedAddress     object
address/city              object
address/zipcode            int64
bathrooms                float64
bedrooms                 float64
                          ...   
yearBuilt                float64
zestimate                float64
zestimateHighPercent     float64
zestimateLowPercent      float64
zpid                       int64
Length: 348, dtype: object

In [58]:
ZLLW_df2=ZLLW_df[[
    'ï»¿abbreviatedAddress',
     'address/city',
     'address/zipcode',
     'bathrooms',
     'bedrooms',
     #'brokerageName',
     'dateSold',
     'daysOnZillow',
     #'description',
     #'favoriteCount',
     'homeStatus',
     'homeType',
     'lastSoldPrice',
     'latitude',
     'livingArea',
     'longitude',
     'lotSize',
     'pageViewCount',
     #'postingContact/name',
     'price',
     #'priceHistory/0/attributeSource/infoString1',
     #'priceHistory/0/attributeSource/infoString2',
     #'priceHistory/0/attributeSource/infoString3',
     #'priceHistory/0/buyerAgent/name',
     #'priceHistory/0/buyerAgent/photo/url',
     #'priceHistory/0/buyerAgent/profileUrl',
     'priceHistory/0/date',
     'priceHistory/0/event',
     'priceHistory/0/postingIsRental',
     'priceHistory/0/price',
     'priceHistory/0/priceChangeRate',
     'priceHistory/0/pricePerSquareFoot',
     #'priceHistory/0/sellerAgent/name',
     #'priceHistory/0/sellerAgent/photo/url',
     #'priceHistory/0/sellerAgent/profileUrl',
     #'priceHistory/0/showCountyLink',
     #'priceHistory/0/source',
     #'priceHistory/0/time',
     #'propertyTaxRate',
     'rentZestimate',
     'resoFacts/atAGlanceFacts/0/factValue',
     'resoFacts/basement', 
     'resoFacts/heating/0',
     'resoFacts/atAGlanceFacts/2/factLabel',
     'resoFacts/atAGlanceFacts/2/factValue',
     'resoFacts/cooling/0',
     'resoFacts/atAGlanceFacts/3/factLabel',
     'resoFacts/atAGlanceFacts/3/factValue',
     'taxAssessedValue',
     'taxAssessedYear',
     'taxHistory/0/taxIncreaseRate',
     'taxHistory/0/taxPaid',
     'taxHistory/0/time',
     'taxHistory/0/value',
     'taxHistory/0/valueIncreaseRate',
     'url',
     'yearBuilt',
     'zestimate',
     'zestimateHighPercent',
     'zestimateLowPercent',
     'zpid'
]].copy()
pd.set_option('display.max_columns', None)
ZLLW_df2.head()

,ï»¿abbreviatedAddress,address/city,address/zipcode,bathrooms,bedrooms,dateSold,daysOnZillow,homeStatus,homeType,lastSoldPrice,latitude,livingArea,longitude,lotSize,pageViewCount,price,priceHistory/0/date,priceHistory/0/event,priceHistory/0/postingIsRental,priceHistory/0/price,priceHistory/0/priceChangeRate,priceHistory/0/pricePerSquareFoot,rentZestimate,resoFacts/atAGlanceFacts/0/factValue,resoFacts/basement,resoFacts/heating/0,resoFacts/atAGlanceFacts/2/factLabel,resoFacts/atAGlanceFacts/2/factValue,resoFacts/cooling/0,resoFacts/atAGlanceFacts/3/factLabel,resoFacts/atAGlanceFacts/3/factValue,taxAssessedValue,taxAssessedYear,taxHistory/0/taxIncreaseRate,taxHistory/0/taxPaid,taxHistory/0/time,taxHistory/0/value,taxHistory/0/valueIncreaseRate,url,yearBuilt,zestimate,zestimateHighPercent,zestimateLowPercent,zpid
0,2860 NE Blossom Hill Rd,Fairview,97024,3.0,3.0,1.620000e+12,25.0,RECENTLY_SOLD,TOWNHOUSE,315400,45.543571,1806.0,-122.441765,1555.0,47.0,315400,5/21/2021,Sold,False,315400,0.487736,175.0,1950.0,Townhouse,Finished,Forced air,Heating,"Forced air, Gas",NaN,Cooling,NaN,273960.0,2020.0,0.043022,2989.74,1.590000e+12,172290.0,0.029950,https://www.zillow.com/homedetails/2860-NE-Blo...,2007.0,317406.0,5.0,5.0,72258538
1,20386 NE Mackenzie Ln,Fairview,97024,3.0,3.0,1.620000e+12,53.0,RECENTLY_SOLD,SINGLE_FAMILY,400000,45.547585,1518.0,-122.453224,3484.0,4.0,400000,4/23/2021,Sold,False,400000,0.066667,264.0,NaN,Single Family Residence,Crawl Space,Forced Air,Heating,Forced Air,Central Air,Cooling,Central Air,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.zillow.com/homedetails/20386-NE-Ma...,2001.0,NaN,NaN,NaN,2073593774
2,1121 SW 10th Dr,Gresham,97080,3.0,4.0,1.620000e+12,11.0,RECENTLY_SOLD,SINGLE_FAMILY,512000,45.488228,2724.0,-122.444420,9583.0,153.0,512000,6/4/2021,Sold,False,512000,0.068894,188.0,1995.0,Ranch,Daylight,Forced Air,Heating,"Forced Air, Fireplace(s)",Wall Unit(s),Cooling,Wall Unit(s),397560.0,2020.0,0.026868,5178.62,1.590000e+12,282340.0,0.029987,https://www.zillow.com/homedetails/1121-SW-10t...,1982.0,512355.0,5.0,5.0,53959026
3,19309 NE Glisan St,Portland,97230,1.0,3.0,1.620000e+12,11.0,RECENTLY_SOLD,SINGLE_FAMILY,348000,45.526634,1217.0,-122.464088,13939.0,32.0,348000,6/4/2021,Sold,False,348000,0.025037,286.0,1695.0,Ranch,Crawl Space,Forced Air,Heating,"Forced Air, Fireplace(s)",NaN,Cooling,NaN,269520.0,2017.0,0.045376,2420.36,1.500000e+12,142330.0,0.029959,https://www.zillow.com/homedetails/19309-NE-Gl...,1953.0,NaN,NaN,NaN,176563865
4,25-65 NW 20th St,Gresham,97030,3.0,6.0,1.620000e+12,14.0,RECENTLY_SOLD,APARTMENT,510000,45.511238,2400.0,-122.431534,8545.0,2.0,510000,6/1/2021,Sold,False,510000,1.020202,213.0,1395.0,Apartment,NaN,Other,Heating,Other,Central,Cooling,Central,421980.0,2020.0,0.026878,2809.61,1.590000e+12,153180.0,0.029989,https://www.zillow.com/homedetails/25-65-NW-20...,1967.0,NaN,NaN,NaN,53819568


In [59]:
ZLLW_df3=ZLLW_df2.rename(columns={"ï»¿abbreviatedAddress": "Address", "address/city": "City","address/zipcode": "Zipcode"})
ZLLW_df3.head(50)

,Address,City,Zipcode,bathrooms,bedrooms,dateSold,daysOnZillow,homeStatus,homeType,lastSoldPrice,latitude,livingArea,longitude,lotSize,pageViewCount,price,priceHistory/0/date,priceHistory/0/event,priceHistory/0/postingIsRental,priceHistory/0/price,priceHistory/0/priceChangeRate,priceHistory/0/pricePerSquareFoot,rentZestimate,resoFacts/atAGlanceFacts/0/factValue,resoFacts/basement,resoFacts/heating/0,resoFacts/atAGlanceFacts/2/factLabel,resoFacts/atAGlanceFacts/2/factValue,resoFacts/cooling/0,resoFacts/atAGlanceFacts/3/factLabel,resoFacts/atAGlanceFacts/3/factValue,taxAssessedValue,taxAssessedYear,taxHistory/0/taxIncreaseRate,taxHistory/0/taxPaid,taxHistory/0/time,taxHistory/0/value,taxHistory/0/valueIncreaseRate,url,yearBuilt,zestimate,zestimateHighPercent,zestimateLowPercent,zpid
0,2860 NE Blossom Hill Rd,Fairview,97024,3.0,3.0,1.620000e+12,25.0,RECENTLY_SOLD,TOWNHOUSE,315400,45.543571,1806.0,-122.441765,1555.0,47.0,315400,5/21/2021,Sold,False,315400,0.487736,175.0,1950.0,Townhouse,Finished,Forced air,Heating,"Forced air, Gas",NaN,Cooling,NaN,273960.0,2020.0,0.043022,2989.74,1.590000e+12,172290.0,0.029950,https://www.zillow.com/homedetails/2860-NE-Blo...,2007.0,317406.0,5.0,5.0,72258538
1,20386 NE Mackenzie Ln,Fairview,97024,3.0,3.0,1.620000e+12,53.0,RECENTLY_SOLD,SINGLE_FAMILY,400000,45.547585,1518.0,-122.453224,3484.0,4.0,400000,4/23/2021,Sold,False,400000,0.066667,264.0,NaN,Single Family Residence,Crawl Space,Forced Air,Heating,Forced Air,Central Air,Cooling,Central Air,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.zillow.com/homedetails/20386-NE-Ma...,2001.0,NaN,NaN,NaN,2073593774
2,1121 SW 10th Dr,Gresham,97080,3.0,4.0,1.620000e+12,11.0,RECENTLY_SOLD,SINGLE_FAMILY,512000,45.488228,2724.0,-122.444420,9583.0,153.0,512000,6/4/2021,Sold,False,512000,0.068894,188.0,1995.0,Ranch,Daylight,Forced Air,Heating,"Forced Air, Fireplace(s)",Wall Unit(s),Cooling,Wall Unit(s),397560.0,2020.0,0.026868,5178.62,1.590000e+12,282340.0,0.029987,https://www.zillow.com/homedetails/1121-SW-10t...,1982.0,512355.0,5.0,5.0,53959026
3,19309 NE Glisan St,Portland,97230,1.0,3.0,1.620000e+12,11.0,RECENTLY_SOLD,SINGLE_FAMILY,348000,45.526634,1217.0,-122.464088,13939.0,32.0,348000,6/4/2021,Sold,False,348000,0.025037,286.0,1695.0,Ranch,Crawl Space,Forced Air,Heating,"Forced Air, Fireplace(s)",NaN,Cooling,NaN,269520.0,2017.0,0.045376,2420.36,1.500000e+12,142330.0,0.029959,https://www.zillow.com/homedetails/19309-NE-Gl...,1953.0,NaN,NaN,NaN,176563865
4,25-65 NW 20th St,Gresham,97030,3.0,6.0,1.620000e+12,14.0,RECENTLY_SOLD,APARTMENT,510000,45.511238,2400.0,-122.431534,8545.0,2.0,510000,6/1/2021,Sold,False,510000,1.020202,213.0,1395.0,Apartment,NaN,Other,Heating,Other,Central,Cooling,Central,421980.0,2020.0,0.026878,2809.61,1.590000e+12,153180.0,0.029989,https://www.zillow.com/homedetails/25-65-NW-20...,1967.0,NaN,NaN,NaN,53819568
5,1518 SE 12th St,Gresham,97080,2.0,3.0,1.620000e+12,14.0,RECENTLY_SOLD,SINGLE_FAMILY,404200,45.487991,1150.0,-122.416184,7000.0,8.0,404200,6/1/2021,Sold,False,404200,0.976528,351.0,1800.0,SingleFamily,NaN,Forced air,Heating,Forced air,NaN,Cooling,NaN,309260.0,2020.0,0.026862,3232.92,1.590000e+12,176260.0,0.029977,https://www.zillow.com/homedetails/1518-SE-12t...,1967.0,397479.0,6.0,5.0,53963177
6,110 NW Willowbrook Ct,Gresham,97030,2.0,3.0,1.620000e+12,25.0,RECENTLY_SOLD,SINGLE_FAMILY,478200,45.498184,2036.0,-122.451332,6969.0,7.0,478200,5/21/2021,Sold,False,478200,0.804528,235.0,2250.0,SingleFamily,Finished,Heat pump,Heating,Heat pump,Central,Cooling,Central,373030.0,2020.0,0.026860,4292.53,1.590000e+12,234030.0,0.029971,https://www.zillow.com/homedetails/110-NW-Will...,1978.0,481241.0,5.0,5.0,53978110
7,4252 SW Lillyben Ave,Gresham,97080,2.0,4.0,1.620000e+12,26.0,RECENTLY_SOLD,SINGLE_FAMILY,530000,45.467598,1947.0,-122.458389,4791.0,69.0,530000,5/20/2021,Sold,False,530000,0.104167,272.0,NaN,Ranch,Crawl Space,Forced Air - 95+%,Heating,"Forced Air - 95+%, Fireplace(s)",Central Air,Cooling,Central Air,426240.0,2020.0,0.024985,4483.12,1.59

In [60]:
ZLLW_df3.isna().sum()

Address                                     0
City                                        0
Zipcode                                     0
bathrooms                                 484
bedrooms                                  770
dateSold                                    0
daysOnZillow                                6
homeStatus                                  0
homeType                                    0
lastSoldPrice                               0
latitude                                   13
livingArea                                465
longitude                                  13
lotSize                                  2890
pageViewCount                           11267
price                                       0
priceHistory/0/date                         0
priceHistory/0/event                        0
priceHistory/0/postingIsRental              0
priceHistory/0/price                        0
priceHistory/0/priceChangeRate              0
priceHistory/0/pricePerSquareFoot 

In [61]:
ZLLW_df3.to_csv("Resources/cleaned_zillow.csv")